## 2. Analysis of the Wikipedia graph

In the first notebook the extraction of the graph structure from the XML dump of the Wikipedia was described. In this notebook the structured data is used to build a graph representation that is suitable for shortest paths analysis. The spark-graphx library is used for building the Wikipedia graph and running algorithms on it.

### Preparation of the Spark session

Imports and configuration of spark.

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-graphx:2.4.5`
import $ivy.`sh.almond::almond-spark:0.6.0`
import org.apache.spark.graphx._
import org.apache.spark.sql._ // for NotebookSparkSession
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

val spark = {
  NotebookSparkSession.builder()
    .progress(false)
    .master("local[*]")
    .config("spark.executor.memory", "2g")
    .config("spark.local.dir", "/data/flachsenberg/tmp/")
    .getOrCreate()
}

import spark.implicits._

def sc = spark.sparkContext

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import org.apache.spark.graphx._

import org.apache.spark.sql._ // for NotebookSparkSession

import org.apache.log4j.{Level, Logger}

spark: SparkSession = org.apache.spark.sql.SparkSession@53e41fe9
import spark.implicits._


defined function sc

### Building the graph

First, the edges determined before are read in and converted into an RDD of suitable format for the spark-graphx library.

In [2]:
val edgeDF = spark.read.load("links.parquet")
val edges = edgeDF.rdd.map(row => Edge(row.getLong(0), row.getLong(1), row.getInt(2)))
edges.cache

edgeDF: DataFrame = [from: bigint, to: bigint ... 1 more field]
edges: org.apache.spark.rdd.RDD[Edge[Int]] = MapPartitionsRDD[7] at map at cmd1.sc:2
res1_2: org.apache.spark.rdd.RDD[Edge[Int]] = MapPartitionsRDD[7] at map at cmd1.sc:2

In [3]:
edges.take(10).foreach(println(_))
val edgeCount = edges.count()
println(s"Edge count: $edgeCount")

Edge(1321276,11135291,1)
Edge(1379743,8033245,1)
Edge(3896518,8033245,1)
Edge(1379751,1676860,1)
Edge(1690953,1676860,1)
Edge(5102412,1676860,0)
Edge(30518,10432444,1)
Edge(47047,10432444,1)
Edge(1379743,10432444,1)
Edge(5556361,10432444,1)
Edge count: 68319408


edgeCount: Long = 68319408L

Second, the nodes determined before are read in and converted into an RDD of suitable format for the spark-graphx library.

In [4]:
val nodeDF = spark.read.load("titles.parquet")
val nodes = nodeDF.rdd.map(row => (row.getLong(0), row.getString(1)))
nodes.cache

nodeDF: DataFrame = [id: bigint, title: string]
nodes: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[15] at map at cmd3.sc:2
res3_2: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[15] at map at cmd3.sc:2

In [5]:
nodes.take(10).foreach(println(_))
val nodeCount = nodes.count()
println(s"Node count: $nodeCount")

(2009958,Zülle)
(2009959,Zülicke)
(2009960,Festung Germersheim)
(2009961,Ludmila Formanová)
(2009963,Zübert)
(2009964,Zötzsche)
(2009965,Ludmila Formanova)
(2009966,Zörgiebel)
(2009967,Zöpfl)
(2009968,Zöpel)
Node count: 4017802


nodeCount: Long = 4017802L

From the edge RDD and the node RDD a graph can be constructed.

In [6]:
val graph = Graph(nodes, edges, "NO-ARTICLE")
graph.cache
val nofNodes = graph.vertices.count()
println(s"Graph vertices: $nofNodes")
val nodeEdges = graph.edges.count()
println(s"Graph edges: $nodeEdges")

Graph vertices: 4017802
Graph edges: 68319408


graph: Graph[String, Int] = org.apache.spark.graphx.impl.GraphImpl@1a181b95
res5_1: Graph[String, Int] = org.apache.spark.graphx.impl.GraphImpl@1a181b95
nofNodes: Long = 4017802L
nodeEdges: Long = 68319408L

In [7]:
// there should be no default vertices
val nofDefaultNodes = graph.vertices.filter({case (_, title) => title == "NO-ARTICLE"}).count()
println(s"Default vertices: $nofDefaultNodes")

Default vertices: 0


nofDefaultNodes: Long = 0L

### Shortest paths analysis

The shortest paths in the Wikipedia graph were analyzed. Here, the analysis was performed with the "Hamburg" page as the source node. We are interested in the shortest path ending at this vertex, so the graph will be inverted below.

In [8]:
val (sourceId, _) = graph.vertices.filter(_._2 == "Hamburg").first

sourceId: VertexId = 2129L

The SSSP problem was solved using the SSSP algorithm given here: https://spark.apache.org/docs/latest/graphx-programming-guide.html#pregel-api. The implementation was modified to not only store the weight of the shortest-path but also allow reconstruction (and therefore also stores the parent on the shortest path).

While not specified explicitly, this algorithm works similar to the Bellman-Ford algorithm (https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm).

In [9]:
// one entry in the result graph stores the minimum distance as well as the parent VertexId on the shortest path.
case class SSSPEntry (parent: VertexId, dist: Double)
// initial entry is infinite distance and no predecessor
val ssspentry0 = SSSPEntry(-1, Double.PositiveInfinity)
// The initial graph consists of infinity distance except for the source vertex.
// Note that we invert the graph because we are interested in paths ending at the given source vertex
val initialGraph = graph.reverse.mapVertices((id, _) =>
    if (id == sourceId) SSSPEntry(-1, 0.0) else ssspentry0)

defined class SSSPEntry
ssspentry0: SSSPEntry = SSSPEntry(-1L, Infinity)
initialGraph: Graph[SSSPEntry, Int] = org.apache.spark.graphx.impl.GraphImpl@48e62c4c

In [10]:
val sssp = initialGraph.pregel(ssspentry0)(
  (id, entry, newEntry) => if (newEntry.dist < entry.dist) newEntry else entry,
  triplet => {
    if (triplet.srcAttr.dist + triplet.attr < triplet.dstAttr.dist) {
      Iterator((triplet.dstId, SSSPEntry(triplet.srcId, triplet.srcAttr.dist + triplet.attr)))
    } else {
      Iterator.empty
    }
  },
  (a, b) => if (a.dist < b.dist) a else b
)
sssp.cache

sssp: Graph[SSSPEntry, Int] = org.apache.spark.graphx.impl.GraphImpl@4cbe3623
res9_1: Graph[SSSPEntry, Int] = org.apache.spark.graphx.impl.GraphImpl@4cbe3623

In [13]:
sssp.vertices.map(_._2.dist).countByValue.toList.sorted

res12: List[(Double, Long)] = List(
  (0.0, 9L),
  (1.0, 68310L),
  (2.0, 2156603L),
  (3.0, 1729296L),
  (4.0, 44785L),
  (5.0, 288L),
  (6.0, 5L),
  (Infinity, 18506L)
)

Interestingly, from only a minority of nodes the node "Hamburg" is not reachable. From every other node, "Hamburg" is reachable within a distance of at most 6.

The following function allows the reconstruction of the shortest path.

In [14]:
def getPathFromTarget(targetId: VertexId, sssp: Graph[SSSPEntry, Int],
                      graph: Graph[String, Int]) : Seq[(String, Double)] = {
  if (targetId == -1) {
    Seq()
  }
  else {
    val (_, entry) = sssp.vertices.filter(_._1 == targetId).first
    val (_, title) = graph.vertices.filter(_._1 == targetId).first
    Seq((title, entry.dist)) ++ getPathFromTarget(entry.parent, sssp, graph)
  }
}

defined function getPathFromTarget

In [16]:
val (targetId, _) = graph.vertices.filter(_._2 == "Zitteraal").first
println(getPathFromTarget(targetId, sssp, graph))

List((Zitteraal,2.0), (Zitteraale,2.0), (Carl von Linné,1.0), (Hamburg,0.0))


targetId: VertexId = 10950018L

This example shows that the distance from node "Zitteraal" (electric eel) to "Hamburg" is 2, i.e. needing 2 clicks. The first node "Zitteraal" is a redirect onto "Zitteraale" which is not counted as a distinct click.

In [17]:
sssp.vertices.filter(_._2.dist == 6.0)
             .collect
             .map({case (id, _) => id})
             .map(x => {
                 getPathFromTarget(x, sssp, graph)
             }).foreach(println(_))

List((Mills Township,6.0), (Mill Township,5.0), (Mill Creek Township,4.0), (Mill Creek Township (Coshocton County, Ohio),3.0), (Township (Vereinigte Staaten),2.0), (Stadt,1.0), (Hamburg,0.0))
List((Tenmile,6.0), (Ten Mile,5.0), (Ten Mile River,4.0), (Ten Mile River (Pazifischer Ozean),3.0), (Mendocino County,3.0), (County (Vereinigte Staaten),2.0), (Département,1.0), (Hamburg,0.0))
List((Alder Creek,6.0), (North Alder Creek,5.0), (North Fork Alder Creek,4.0), (Mendocino County,3.0), (County (Vereinigte Staaten),2.0), (Département,1.0), (Hamburg,0.0))
List((Baker Crossroads,6.0), (Bakers Crossroads,5.0), (Bakers Crossing,4.0), (Bakers,3.0), (Piet Bakers,2.0), (Niederländische Fußballnationalmannschaft,1.0), (Hamburg,0.0))
List((Lerdo,6.0), (Lerdo de Tejada,5.0), (Sebastián Lerdo de Tejada (Begriffsklärung),4.0), (Sebastián Lerdo de Tejada,3.0), (Kaiserreich Mexiko (1864–1867),2.0), (Édouard Manet,1.0), (Hamburg,0.0))


Finally, all shortest path with distance 6 are shown. Interestingly, these have a similar structure, i.e. there is a chain of similar sounding articles in the beginning of each path.